# Notebook 2_a

In [1]:
import pickle
from pathlib import Path
import tdt
import pandas as pd 
import papermill as pm

In [2]:
import Notebook_2_helper_functions as Nb_2

In [3]:
mouse = 'T240'
data_day = '2019_09_27__T240'
date = '2019_09_27'

photo_day = 'T240-145905'

HowManyBack=1 

data_dir_input = "/Users/gilmandelbaum/Desktop/T240_mouse"
data_dir_output = "/Users/gilmandelbaum/Desktop/output/"

In [4]:
# Parameters
data_dir_input = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/raw_data_final"
data_dir_output = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/2.raw_data_final_output_new_version/"
drop2blocks = "no"
drop2lastblocks = "no"
seq_str = "0a1b2a3a4abcde5a6a7b"
HowManyBack = 1
Nb0 = "a"
Nb1 = "b"
Nb2 = "a"
Nb3 = "a"
Nb4 = ["a", "b", "c", "d", "e"]
Nb5 = "a"
Nb6 = "a"
Nb7 = "b"
Tags = [
    "Reward_NoReward_tag",
    "ENL_tag",
    "Cue_to_Sel_tag",
    "Cue_to_Sel_fromPrevTrial_tag",
    "firstENLp_tag",
]
Timing = [600, 200, 15, 15, 20]
Start_or_end = ["start", "end", "start", "start", "start"]
y_axis_list = ["df_f", "z_score", "z_score_sliding_window"]
mouse = "T163"
data_day = "2018_12_04__T163"
date = "2018_12_04"
photo_day = "T163-114628"


# Import behavior related data per state transition

In [5]:
behavior_data_states = pd.read_csv(data_dir_input+"/"+mouse+"/"+data_day+"/"+mouse+"_dataAnalog.csv")

In [6]:
behavior_data_states = Nb_2.prepare_behavior_data(behavior_data_states)

# Import photometry data


In [7]:
photometry_data = tdt.read_block(data_dir_input+"/"+mouse+"/"+photo_day)

read from t=0s to t=1301.59s


# Grab only what is needed from the TDT photo tanks and make a df. 

In [ ]:
#The photometry TDT data structure has a lot of info. 
#We will create 1 data frame that has: 
# 
#['d2 R' , 'd1 R', 'd2 L', "d1 L", "toBehSys", "fromBehSys" ]

def reading_in_photometryData_all_Channels(data):
    photometry_all_channels_d = {}
    photometry_all_channels_d["d2 R"]=photometry_data.streams.grn1.data 
    photometry_all_channels_d["d1 R"]=photometry_data.streams.red1.data 
    
    
    photometry_all_channels_d["d2 L"]=photometry_data.streams.grn2.data 
    photometry_all_channels_d["d1 L"]=photometry_data.streams.red2.data 
    
    
    
    photometry_all_channels_d["toBehSys"]=photometry_data.streams.toGG.data 
    photometry_all_channels_d["fromBehSys"]=photometry_data.streams.froG.data 
    photometry_all_channels= pd.DataFrame(data=photometry_all_channels_d)
    
    return (photometry_all_channels)

In [ ]:
photometry_all_channels=reading_in_photometryData_all_Channels(photometry_data)

# Clean the photometry data (or any neural data)

In [ ]:
photometry_all_channels_clean = Nb_2.handshake_behav_recording_sys(photometry_all_channels)

# Clean behavior states data 

In [ ]:
behavior_data_states_clean = Nb_2.define_behavior_data_states(behavior_data_states)

# pre syncing steps for photometry data (or any neural data) and behavior data 

In [ ]:
behavior_data_states_clean_binNumberlist=Nb_2.number_of_bins_beh_data_per_trial(behavior_data_states_clean)

In [ ]:
photometry_all_channels_clean_binNumberlist = Nb_2.number_of_bins_photo_data_per_trial(photometry_all_channels_clean)

# Resample the photometry data (or any neural data) and align to 200 hz state transition behavioral data 

In [ ]:
aligned_behav_photo = Nb_2.resample_photoData_and_align_photo_with_beh(behavior_data_states_clean_binNumberlist,
                                                            behavior_data_states_clean,
                                                            photometry_all_channels_clean,
                                                            photometry_all_channels_clean_binNumberlist)

In [ ]:
aligned_behav_photo = Nb_2.tag_real_full_trial(aligned_behav_photo)

In [ ]:
aligned_behav_photo.to_csv("aligned_behav_photo.csv")

In [ ]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_2_a.pickle"
my_path = root / d 
my_file = open(my_path, 'wb')
my_file = pickle.dump(aligned_behav_photo,my_file)